# Task
- [x] 课程回顾（二分法，牛顿法，梯度下降法）
- [x] 冒泡排序
- [x] 快速排序

## 二分法

二分法的思路是每次排除一半样本的试错方法，把样本一分为二（A和B），那么目标值不在A就在B里，不断缩小范围

In [8]:
import math

def binary_sqrt(n):
    epsilon = 1e-10         # quit flag
    start = 0
    end = n
    mid = start + (end - start) / 2
    diff = mid ** 2 - n
    while abs(diff) >= epsilon:
        # 值过大则尝试小的一半，否则就尝试大的一半，修改边界值即可
        if diff > 0:
            end = mid
        else:
            start = mid
        mid = start + (end - start) / 2
        diff = mid ** 2 - n
    return mid

for i in range(1,11):
    print(f'estimated:\t{binary_sqrt(i)}, \t sqrt({i}): \t {math.sqrt(i)}')

estimated:	0.9999999999708962, 	 sqrt(1): 	 1.0
estimated:	1.4142135623842478, 	 sqrt(2): 	 1.4142135623730951
estimated:	1.7320508075645193, 	 sqrt(3): 	 1.7320508075688772
estimated:	2.0, 	 sqrt(4): 	 2.0
estimated:	2.2360679775010794, 	 sqrt(5): 	 2.23606797749979
estimated:	2.449489742779406, 	 sqrt(6): 	 2.449489742783178
estimated:	2.6457513110653963, 	 sqrt(7): 	 2.6457513110645907
estimated:	2.8284271247393917, 	 sqrt(8): 	 2.8284271247461903
estimated:	2.999999999989086, 	 sqrt(9): 	 3.0
estimated:	3.162277660157997, 	 sqrt(10): 	 3.1622776601683795


## 牛顿法

牛顿法用的是斜率的思想，对$f(x)=0$，选一个足够接近目标值($x$)的点($x_0$)，计算其切线与X轴的交点($x_1$），这个交点往往比$x_1$更接近$x$，数次迭代后肯定越来越接近目标值$x$。  

$f'(x_0) = \frac{f(x_0)-f(x_1)}{x_0-x_1}$    
$\because f(x_1)=0\ \Rightarrow x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$  

而求任意正整数$a$的平方根，函数就变成了 $f(x) = a => f(x) = x^2 - a = 0, f'(x) = 2x$，迭代用如下公式：
$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} = x_n - \frac{x_n^2 - a}{2x_n}$

In [14]:
def newton_sqrt(n):
    x_n = n / 2
    epsilon = 1e-10             # quit flag
    
    f_x = lambda a : a**2 - n   # f(x)=x^2 - a
    df_x = lambda a : 2*a       # derivative of f(x)
    x_next = lambda a : a - f_x(a) / df_x(a)
    
    while abs(x_n ** 2 - n) > epsilon:
        x_n = x_next(x_n)
    return x_n

for i in range(1, 10):
    print(f'sqrt({i})\t{newton_sqrt(i)}')

sqrt(1)	1.000000000000001
sqrt(2)	1.4142135623746899
sqrt(3)	1.7320508075688772
sqrt(4)	2.0
sqrt(5)	2.23606797749979
sqrt(6)	2.4494897427831788
sqrt(7)	2.6457513110646933
sqrt(8)	2.8284271247493797
sqrt(9)	3.0


## 梯度下降法

梯度下降法的数学原理是$f(x_1,x_2,\dots$)的gradient（$\nabla f$）就是其最陡爬升方向（`steepest ascent`），在一元方程里就是过某点的斜率（`slope`)，或者说函数的导数（`derivative`），我们要到局部最小值，显然就应该向相向的方向走。并且由于越接近目标值（谷底），斜率越小，所以即使我们选择一个固定的步长（`learning rate`），也是会有一个越来越小的步进值去逼近极值，而无需刻意去调整步长。

以上是思路，它$\color{red}{并不是作用到要求的函数本身}$上去的，而是一个一般用`最小二乘法`来构造的二次函数$e(x) = \frac{1}{2}(f(x) - Y)^2$。

$e(x)$表示的是不同的x取值下与目标值$Y$的差的平方（有时叫损失函数*loss*），既然是一个简单二次函数，就能求极值，且它的最小值意味着当x值为该值时估算原函数$f(x)=Y$的**误差最小**，所以对函数$f(x)$，我们构造梯度下降法时用的应该是表示误差的最小二乘法二次函数，设为$e(x)$，有：

$e(x) = \frac{1}{2}(f(x) - Y)^2$  (1/2的作用仅仅是为了取导数时消除常数项，简化计算)   
$e'(x) = (f(x) - Y) \cdot f'(x) = \Delta y \cdot f'(x)\quad \color{green}{\Leftarrow Chain\ Rule}$   
$\Delta x = e'(x) \cdot lr = \Delta y \cdot f'(x) \cdot lr\ \color{red}{\Leftarrow这就是课程里公式的由来}$    
$x_{n+1} = x_n - \Delta x = x_n - \Delta y \cdot f'(x) \cdot lr$

这时可以写代码了

In [10]:
def gradient_sqrt(n):
    x       = n / 2       # first try
    lr      = 0.01        # learning rate
    epsilon = 1e-10       # quit flag
    
    f_x     = lambda a : a**2
    df_dx   = lambda a : 2*a
    delta_y = lambda a : f_x(a) -n
    e_x     = lambda a : delta_y(a)**2 * 0.5     # funcon of loss
    de_dx   = lambda a : delta_y(a) * df_dx(a)   # derivative of loss
    delta_x = lambda a : de_dx(a) * lr
    
    count   = 0
    while abs(x**2 - n) > epsilon:
        count += 1
        x = x - delta_x(x)
    return x, count

for i in range(1, 10):
    print(f'sqrt({i}): {gradient_sqrt(i)}次')

sqrt(1): (0.9999999999519603, 593)次
sqrt(2): (1.4142135623377403, 285)次
sqrt(3): (1.7320508075423036, 181)次
sqrt(4): (2.0, 0)次
sqrt(5): (2.236067977522142, 103)次
sqrt(6): (2.449489742798969, 87)次
sqrt(7): (2.645751311082885, 73)次
sqrt(8): (2.828427124761154, 63)次
sqrt(9): (3.00000000001166, 55)次


> 思考：
>
> 牛顿法与梯度下降法思路是相反的，牛顿法是用线性方程的根来逼近非线性方程的根，而梯度下降法恰恰是在一个刻意构造的**非线性的**二次函数的曲线里求极值。

## 冒泡排序

冒泡排序基础原理是每一轮都让最大的值移到最右边，如果想小优化可以在每一轮过后都把最后一个（已经是最大的值）排除出去。而且看到了有同学还有更优化的算法，不是每一轮都从头比到尾，同样体量的两万个随机数，我的代码用了80秒他的用了40秒

但同样体量的随机数用快速排序只花了0.13秒！

In [8]:
def bubble_sort(arr):
    for i in range(1, len(arr)):
        for j in range(len(arr)-i):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
bubble_sort([0, 2, 1, 3, 5, 1, 1])

[0, 1, 1, 1, 2, 3, 5]

## 快速排序

快速排序我理解为二分法的一种应用，选出一个合适的（或任意的）中值(`pivot`），把比它大的和小的分列到两边，再对两边进行上述分类的递归操作。

这里用一个从两侧压缩的快速排序法，交替从左右把大于和小于中值的放两边，每查询一轮边界就会压缩很多，看如下演示：

随便写个数组[6,7,3,2,14,9]，任取一个数为pivot，就第1个吧（6），  
- 左箭头表示从右往左找第一个小于pivot的值，右箭头表示从左往右找第一个大于pivot的值  
- 红色代表标红位，废位，即当前位找到本轮符合要求的值，但挪到两侧去了，$\color{red}{下一轮的符合条件的值应该放入这个标红位里}$
- 括号里的表示是这一轮该位置赋的新值，它来自于标红位，同时，括号的位置也就是上一轮的标红位
- 划掉的表示已经压缩了左右边界，下一轮就不要在这些数里面选了（为了视觉简洁，标红位就不划了）
$
\require{cancel}
\begin{array}{c|cccccc|l}
index&0&1&2&3&4&5&\\
\hline
array&\color{red}6&7&3&2&14&9\\
\underleftarrow{\small找小数}&\cancel{(2)}&7&3&\color{red}2&\cancel{14}&\cancel{9}&找到2，放到索引0\\
\underrightarrow{\small找大数}&\cancel{2}&\color{red}7&3&(7)&\cancel{14}&\cancel{9}&找到7，放到索引3\\
\underleftarrow{\small找小数}&\cancel{2}&(3)&\color{red}3&\cancel{7}&\cancel{14}&\cancel{9}&找到3，放到索引2\\
&2&3&(6)&7&14&9&(1,2)索引间已没有大于6的数，排序完成，回填6
\end{array}
$

接下来用同样的逻辑递归6左边的`[2]`和右边的`[7,14,9]`排序即可


In [2]:
def q_sort(array, start, end):
    # （left， right）用来保存不断缩小的查找数组索引界限
    left, right = start, end
    index = start
    pivot = array[start]
    
    while left < right:
#         print(array)
        # 从右往左选小于pivot的数
        matched = False # 标识这一轮有没有找到合适的数（如果没找到其实说明排序已经完成）
        for i in reversed(range(left+1, right+1)): # 去头，含尾, 反序
            if array[i] <= pivot:
                array[index] = array[i]
                right = i  # 从右到左比到第i个才有比pivot小的数，那么i右侧全大于pivot，下次可以缩小范围了
                index = i
                matched = True
                break
        if not matched:
            break  # 右侧没有找到更小的数，说明剩余数组全是大数，已经排完了
            
        left += 1 # 找到了填入新数后就顺移一位
        matched = False
        # 从左往右选大于pivot的数
        for i in range(left, right): # 有头无尾
            if array[i] > pivot:
                array[index] = array[i]
                left = i # 此时i左侧也没有比pivot大的数，下次再找也可以忽略了，也标记下缩小范围
                index = i
                matched = True
                break;
        if not matched:
            break
        right -= 1
    array[index] = pivot # 把标红位设为pivot
    
    # 开始递归处理左右切片
    if start < index-1:
        q_sort(array, start, index-1)
    if end > index+1:
        q_sort(array, index+1, end)

    return array

arr = [0, 2, 1, 3, 5, 1, 1]
q_sort(arr, 0, len(arr)-1)

[0, 1, 1, 1, 2, 3, 5]

In [5]:
import numpy as np
import time
start_time = time.time()
np.random.seed(7)
length = 20000
array_list = list(np.random.randint(0, 100000, size=(length,)))

arr = q_sort(array_list, 0, length-1)
for i in range(5):
    print(f'round {i+1} cost time:\t{time.time() - start_time:.5f}')
print(arr[:100])

round 1 cost time:	0.12985
round 2 cost time:	0.12999
round 3 cost time:	0.13001
round 4 cost time:	0.13003
round 5 cost time:	0.13004
[0, 0, 10, 11, 14, 14, 14, 16, 18, 20, 22, 25, 31, 42, 73, 74, 76, 83, 87, 88, 106, 119, 121, 121, 123, 124, 126, 128, 136, 138, 142, 143, 143, 144, 148, 153, 153, 169, 171, 180, 192, 196, 219, 224, 225, 236, 240, 254, 264, 271, 274, 274, 274, 278, 280, 290, 291, 299, 306, 310, 316, 323, 326, 333, 334, 336, 341, 345, 346, 346, 356, 359, 359, 363, 381, 381, 382, 394, 396, 396, 407, 416, 418, 425, 430, 435, 447, 460, 463, 466, 466, 472, 476, 477, 482, 487, 488, 491, 497, 498]
